In [22]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [23]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
   name="rag", dimension = 1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1") #num of dimensions openai has, needs to match otherwise errors <3 ; metric is for search; ran on AWS
)
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY2"))
pc.create_index(
    name="players", dimension = 1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1") #num of dimensions openai has, needs to match otherwise errors <3 ; metric is for search; ran on AWS
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '26ee920777e78161ec9b6f0d7183b55a', 'Date': 'Sat, 24 Aug 2024 02:58:34 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [24]:
import json
data = json.load(open("stats.json"))
data['stats']


[{'player_name': 'PRX f0rsakeN',
  'most_used_hero': 'Jett',
  'recent_performance': {'matches_played': 100,
   'wins': 65,
   'average_kda': '2.5'},
  'top_used_weapon': 'Vandal',
  'stream_schedule': {'streams_regularly': True,
   'days': ['Monday', 'Wednesday', 'Friday'],
   'time': '7:00 PM - 11:00 PM'}},
 {'player_name': 'NAVI ardiis',
  'most_used_hero': 'Killjoy',
  'recent_performance': {'matches_played': 90,
   'wins': 58,
   'average_kda': '2.3'},
  'top_used_weapon': 'Phantom',
  'stream_schedule': {'streams_regularly': False, 'days': [], 'time': ''}},
 {'player_name': 'FNATIC Chronicle',
  'most_used_hero': 'Sova',
  'recent_performance': {'matches_played': 85,
   'wins': 60,
   'average_kda': '2.8'},
  'top_used_weapon': 'Operator',
  'stream_schedule': {'streams_regularly': True,
   'days': ['Tuesday', 'Thursday'],
   'time': '8:00 PM - 12:00 AM'}},
 {'player_name': 'LEV aspas',
  'most_used_hero': 'Reyna',
  'recent_performance': {'matches_played': 95,
   'wins': 70,
   

In [25]:
import json
data2 = json.load(open("agents.json"))
data2['agents']

[{'name': 'Brimstone',
  'abilities': [{'name': 'Incendiary',
    'type': 'Grenade',
    'description': 'Launches an incendiary grenade that deploys a damaging field of fire.'},
   {'name': 'Stim Beacon',
    'type': 'Buff',
    'description': 'Deploys a beacon that gives all nearby players a rapid-fire buff.'},
   {'name': 'Sky Smoke',
    'type': 'Smoke',
    'description': 'Deploys a tactical map allowing Brimstone to call in orbital smokescreens that obscure vision.'},
   {'name': 'Orbital Strike',
    'type': 'Ultimate',
    'description': 'Calls in a powerful orbital strike that deals high damage over time in a large area.'}]},
 {'name': 'Jett',
  'abilities': [{'name': 'Cloudburst',
    'type': 'Smoke',
    'description': 'Throws a projectile that expands into a brief vision-blocking cloud on impact.'},
   {'name': 'Updraft',
    'type': 'Movement',
    'description': 'Propels Jett high into the air.'},
   {'name': 'Tailwind',
    'type': 'Movement',
    'description': 'Dashes J

In [26]:
processed_data = []
client = OpenAI()

for stat in data['stats']:
    response = client.embeddings.create(
        input=stat['player_name'],
        model="text-embedding-ada-002"
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": stat["player_name"],
        "metadata": {
            "most_used_hero": stat["most_used_hero"],
            "average_kda": stat["recent_performance"]["average_kda"],
            "top_used_weapon": stat["top_used_weapon"],
            "streams_regularly": str(stat["stream_schedule"]["streams_regularly"]),  # Convert boolean to string
            "stream_days": ", ".join(stat["stream_schedule"]["days"]),  # Flatten list into a string
            "stream_time": stat["stream_schedule"]["time"]
        }
    })


In [27]:
processed_data2 = []
client = OpenAI()

for agents in data2['agents']:
    response = client.embeddings.create(
        input=agents['name'],
        model='text-embedding-ada-002'
    )
    embedding = response.data[0].embedding
    abilities_json = json.dumps(agents["abilities"])
    processed_data2.append({
        "values": embedding,
        "id": agents["name"],
        "metadata": {
            "abilities": abilities_json
        }
    })
    

In [ ]:
processed_data[2]


In [28]:
index = pc.Index('rag')
index.upsert( #insert into db
        vectors = processed_data,
        namespace = "ms1" #data in vector db for user id, in firebase, index is collection, namespace is document    
) 



{'upserted_count': 10}

In [29]:
index=pc.Index('players')
index.upsert(
        vectors=processed_data2,  # This should only contain the values, id, and metadata
        namespace2="agents"  # Pass the namespace as an argument to upsert
)


{'upserted_count': 11}

In [30]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 11}, 'agents': {'vector_count': 11}},
 'total_vector_count': 22}